<a href="https://colab.research.google.com/github/Roberto014/Anal-tica-de-datos-y-herramientas-de-inteligencia-artificial-II/blob/main/Act%203.6%20Regresi%C3%B3n%20No%20Lineal/Act_3_6_CODIGO_DASBOARDRegresi%C3%B3n_No_Lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install streamlit

In [ ]:
%pip install plotly

In [ ]:
%pip install funpymodeling

In [ ]:
#Instalamos el tunnel local (node.js)
#Para instalar npm en visual studio
#1. Desde google escribir node.js
#2. Instalar la versión mas recomendada
!npm install localtunnel


up to date, audited 23 packages in 854ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [18]:
%%writefile app.py
# Verificamos que todas las librerías se puedan importar
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from funpymodeling.exploratory import freq_tbl
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

# Definimos la instancia de streamlit
@st.cache_resource

# Función para cargar datos de México
def cargar_datos_mexico():
    # Carga de datos desde un archivo CSV
    datos = pd.read_csv('Datos_limpios (4).csv', encoding='utf-8', index_col=0)

    # Procesamiento de datos
    datos_numericos = datos.copy()

    # Eliminamos columnas que no se necesitan
    datos_numericos.drop(['last_scraped', 'source', 'name', 'description', 'host_name',
                          'host_since', 'host_location', 'host_response_time',
                          'host_verifications', 'host_has_profile_pic',
                          'host_identity_verified', 'neighbourhood_cleansed',
                          'property_type', 'bathrooms_text', 'amenities',
                          'has_availability', 'calendar_last_scraped',
                          'first_review', 'last_review', 'host_is_superhost',
                          'room_type', 'instant_bookable'], axis=1, inplace=True)

    # Seleccionamos solo columnas numéricas
    datos_numericos = datos_numericos.select_dtypes(include=[np.number])

    # Calculamos los coeficientes de correlación
    coef_correlacion = abs(datos_numericos.corr())

    # Variables a analizar
    resultados_datos = datos_numericos[['host_response_rate', 'host_acceptance_rate',
                                         'host_total_listings_count', 'accommodates',
                                         'reviews_per_month', 'price']]

    # Columnas numéricas para el análisis
    datos_numericos_final = datos_numericos.select_dtypes(['float', 'int'])
    columnas_numericas = datos_numericos_final.columns

    # Cargamos el DataFrame original para análisis de variables categóricas
    datos_categoricos = datos.copy()

    return datos, datos_numericos, coef_correlacion, resultados_datos, datos_numericos_final, columnas_numericas, datos_categoricos

# Cargamos los datos
datos, datos_numericos, coef_correlacion, resultados_datos, datos_numericos_final, columnas_numericas, datos_categoricos = cargar_datos_mexico()

# Definimos funciones para modelos no lineales
def funcion_cuadratica(x, a, b, c):
    return a * x**2 + b * x + c

def funcion_exponencial(x, a, b, c):
  return a*np.exp(-b*x) + c

def funcion_lineal_con_producto(x, a, b, c):
    return (a * x**2 + b) / c * x

# Página principal - Título
st.title('ANÁLISIS DE DATOS')
st.header('Regresión No Lineal')

# Análisis para la Ciudad de México
st.subheader('Análisis para la Ciudad de México')

# Checkbox para mostrar el heatmap
heatmap_checkbox = st.sidebar.checkbox(label="Mostrar Heatmap")

if heatmap_checkbox:
    # Generamos un Heatmap para todas las variables
    heatmap = px.imshow(coef_correlacion, text_auto=True,
                        title="Mapa de Calor de Correlación", width=800, height=600)
    st.plotly_chart(heatmap)

# Checkbox para mostrar la matriz de correlación
matriz_checkbox = st.sidebar.checkbox(label="Mostrar matriz de correlación")

if matriz_checkbox:
    # Mostramos la matriz de correlación
    st.write(coef_correlacion)

# Selector de variables objetivo
st.sidebar.text('Seleccione una Variable Objetivo para graficar')
variable_objetivo_seleccionada = st.sidebar.selectbox(label='Variables Objetivo', options=resultados_datos.columns)

st.sidebar.subheader('Criterios de Graficación')
variable_x = st.sidebar.selectbox(label='Seleccione Variable X', options=columnas_numericas)

st.sidebar.subheader('Función para Graficar')
funcion_seleccionada = st.sidebar.selectbox(label='Seleccione Funcion para graficar', options=['Cuadratica', 'Exponencial', 'Función Lineal con Producto'])

if funcion_seleccionada == 'Cuadratica':
    copia_datos = datos_numericos.copy()

    # Variables dependientes e independientes
    x = datos_numericos[[str(variable_x)]]
    y = datos_numericos[[str(variable_objetivo_seleccionada)]]

    st.text('Función Cuadrática: y = ax^2 + bx + c')

    try:
        # Ajustamos los parámetros de la función
        parametros, covs = curve_fit(funcion_cuadratica, datos_numericos[variable_x], datos_numericos[variable_objetivo_seleccionada], maxfev=5000)

        # Predicciones
        y_pred = funcion_cuadratica(x, *parametros)
        copia_datos['y_pred'] = y_pred

        # Coeficiente de determinación
        r2 = r2_score(y, y_pred)
        r = np.sqrt(abs(r2))

        st.text('Coeficiente de Correlación:')
        st.text(str(r))
        st.text('Coeficiente de Determinación (R²):')
        st.text(str(r2))

        # SCATTERPLOT
        figura_rnl = px.scatter(data_frame=copia_datos, x=variable_x, y=[variable_objetivo_seleccionada, 'y_pred'],
                                 title="Regresión No Lineal - Cuadrática")
        st.plotly_chart(figura_rnl)
    except:
        st.text('Error al graficar la función cuadrática')

elif funcion_seleccionada == 'Exponencial':
    copia_datos = datos_numericos.copy()

    # Variables dependientes e independientes
    x = datos_numericos[[str(variable_x)]]
    y = datos_numericos[[str(variable_objetivo_seleccionada)]]

    st.text('Función Exponencial: y = a * exp(-bx) + c')

    try:
        # Ajustamos los parámetros de la función
        parametros, covs = curve_fit(funcion_exponencial, datos_numericos[variable_x], datos_numericos[variable_objetivo_seleccionada], maxfev=5000)

        # Predicciones
        y_pred = funcion_exponencial(x, *parametros)
        copia_datos['y_pred'] = y_pred

        # Coeficiente de determinación
        r2 = r2_score(y, y_pred)
        r = np.sqrt(abs(r2))

        st.text('Coeficiente de Correlación:')
        st.text(str(r))
        st.text('Coeficiente de Determinación (R²):')
        st.text(str(r2))

        # SCATTERPLOT
        figura_rnl = px.scatter(data_frame=copia_datos, x=variable_x, y=[variable_objetivo_seleccionada, 'y_pred'],
                                 title="Regresión No Lineal - Exponencial")
        st.plotly_chart(figura_rnl)
    except:
        st.text('Error al graficar la función exponencial')

elif funcion_seleccionada == 'Función Lineal con Producto':
    copia_datos = datos_numericos.copy()

    # Variables dependientes e independientes
    x = datos_numericos[[str(variable_x)]]
    y = datos_numericos[[str(variable_objetivo_seleccionada)]]

    st.text('Función: y = (a*x^2 + b) / (c*x)')

    try:
        # Ajustamos los parámetros de la función
        parametros, covs = curve_fit(funcion_lineal_con_producto, datos_numericos[variable_x], datos_numericos[variable_objetivo_seleccionada], maxfev=5000)

        # Predicciones
        y_pred = funcion_lineal_con_producto(x, *parametros)
        copia_datos['y_pred'] = y_pred

        # Coeficiente de determinación
        r2 = r2_score(y, y_pred)
        r = np.sqrt(abs(r2))

        st.text('Coeficiente de Correlación:')
        st.text(str(r))
        st.text('Coeficiente de Determinación (R²):')
        st.text(str(r2))

        # SCATTERPLOT
        figura_rnl = px.scatter(data_frame=copia_datos, x=variable_x, y=[variable_objetivo_seleccionada, 'y_pred'],
                                 title="Regresión No Lineal - Función Lineal con Producto")
        st.plotly_chart(figura_rnl)
    except:
        st.text('Error al graficar la función lineal con producto')

Overwriting app.py


In [19]:
!streamlit run app.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.73.111.126


your url is: https://wicked-symbols-pick.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.73.111.126:8501

/content/app.py:143: OptimizeWarning:

Covariance of the parameters could not be estimated

/content/app.py:143: OptimizeWarning:

Covariance of the parameters could not be estimated

  Stopping...
